<a href="https://colab.research.google.com/github/Aravind8281/Natural_language_Processing/blob/main/HuggingFaceTransformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install glove-python-binary


ERROR: Could not find a version that satisfies the requirement glove-python-binary (from versions: none)
ERROR: No matching distribution found for glove-python-binary


In [2]:
import numpy as np
from collections import defaultdict

def build_co_occurrence_matrix(corpus, window_size):
    co_occurrence_matrix = defaultdict(lambda: defaultdict(int))

    for sentence in corpus:
        for i, target_word in enumerate(sentence):
            for j in range(max(0, i - window_size), min(len(sentence), i + window_size + 1)):
                if i != j:
                    context_word = sentence[j]
                    co_occurrence_matrix[target_word][context_word] += 1

    return co_occurrence_matrix

def glove(co_occurrence_matrix, vector_size=50, learning_rate=0.05, epochs=100):
    words = list(co_occurrence_matrix.keys())
    word_vectors = {word: np.random.rand(vector_size) for word in words}
    context_vectors = {word: np.random.rand(vector_size) for word in words}
    biases_target = {word: 0.0 for word in words}
    biases_context = {word: 0.0 for word in words}

    for epoch in range(epochs):
        for target_word, context_words in co_occurrence_matrix.items():
            for context_word, co_occurrence_count in context_words.items():
                weight = (co_occurrence_count / 1.0) ** 0.75  # Weighting function

                dot_product = np.dot(word_vectors[target_word], context_vectors[context_word])
                diff = dot_product + biases_target[target_word] + biases_context[context_word] - np.log(co_occurrence_count)

                # Update word vectors and biases
                word_vectors[target_word] -= learning_rate * weight * diff * context_vectors[context_word]
                context_vectors[context_word] -= learning_rate * weight * diff * word_vectors[target_word]
                biases_target[target_word] -= learning_rate * weight * diff
                biases_context[context_word] -= learning_rate * weight * diff

    return word_vectors

# Example Usage:
corpus = [['I', 'love', 'natural', 'language', 'processing'],
          ['Word', 'embeddings', 'capture', 'semantic', 'relationships']]
window_size = 2

co_occurrence_matrix = build_co_occurrence_matrix(corpus, window_size)
word_vectors = glove(co_occurrence_matrix)

# Print the learned word vectors
for word, vector in word_vectors.items():
    print(f'{word}: {vector}')


I: [ 0.14003308  0.05708992  0.39090111  0.24226452  0.06871776 -0.1714016
  0.19590571  0.10193141  0.12053496  0.42469283 -0.29526412  0.31867111
  0.11851844 -0.14479836  0.12715528 -0.26002005  0.32327358  0.37558473
  0.29376693 -0.06082731  0.21313449  0.42817007 -0.07639968  0.20024588
  0.3561286   0.4015753   0.27624277  0.2706164   0.32220745 -0.16515627
  0.45230808  0.0154566   0.15491515  0.10357507  0.35596462  0.38673023
 -0.03716192  0.04832575 -0.16096503 -0.02762131  0.50730806 -0.12340803
  0.20269973  0.02319566  0.29857068 -0.03611924  0.08850283  0.19669425
  0.39858046  0.04696966]
love: [ 0.08673837 -0.11315111  0.02127496  0.54889929  0.2450109   0.43134984
 -0.14258063 -0.14800036 -0.06588228  0.06547749  0.09570525  0.32309331
  0.37937125  0.39206739 -0.18769158 -0.18746802  0.31059909 -0.12895414
  0.0992413   0.16589148  0.29483338  0.02348266  0.18554899  0.05091507
  0.60698591  0.59076255  0.35392501  0.22455136 -0.23592629 -0.07968451
  0.53983798  0.3

# Hugging Face Transformer




# Loading Pre trained model

In [4]:
from transformers import BertTokenizer,BertModel
model_name="bert-base-uncased"
tokenizer=BertTokenizer.from_pretrained(model_name)
model=BertModel.from_pretrained(model_name)
text = "Hugging Face Transformers is awesome!"
inputs=tokenizer(text,return_tensors="pt")
outputs=model(**inputs)
last_hidden_states = outputs.last_hidden_state
print(last_hidden_states)

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

tensor([[[-0.0294,  0.0961,  0.3038,  ..., -0.2170,  0.0541,  0.1638],
         [ 0.0758, -0.1958,  1.0937,  ..., -0.1868,  0.4230,  0.3861],
         [ 0.2605,  0.1520,  0.7711,  ..., -0.2959,  0.2909,  0.1652],
         ...,
         [ 0.6714,  0.0906, -0.1301,  ..., -0.2988, -0.1300, -0.0532],
         [-0.0525, -0.1482, -0.1025,  ...,  0.7918, -0.0917, -0.1720],
         [ 0.8231,  0.1796, -0.0101,  ...,  0.3322, -0.7087, -0.4356]]],
       grad_fn=<NativeLayerNormBackward0>)


# BERT Transformer

In [5]:
from transformers import pipeline
qa_model = pipeline('question-answering', model='bert-large-uncased-whole-word-masking-finetuned-squad', tokenizer='bert-large-uncased-whole-word-masking-finetuned-squad')
context = "Hugging Face is a great platform for natural language processing."
question = "What is Hugging Face?"
answer=qa_model(question=question,context=context)
print(answer)

config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

{'score': 0.6023051738739014, 'start': 16, 'end': 64, 'answer': 'a great platform for natural language processing'}


# GPT

GPT (Generative Pre-trained Transformer) is a type of transformer-based language model developed by OpenAI. It is designed to generate coherent and contextually relevant text based on the input it receives. GPT is pre-trained on a vast corpus of diverse text data, enabling it to understand language patterns and generate human-like text

In [8]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
model="gpt2"
tokenizer=GPT2Tokenizer.from_pretrained(model)
model=GPT2LMHeadModel.from_pretrained(model)
text="Quick fox jumps over a wall"
input_ids=tokenizer.encode(text,return_tensors="pt")
output = model.generate(input_ids, max_length=50, num_beams=5, no_repeat_ngram_size=2, top_k=50, top_p=0.95, temperature=0.7)
print(output)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


tensor([[21063, 21831, 18045,   625,   257,  3355,    13,   198,   198,     1,
            40,  1101,   407,  1016,   284,  1309,   345,   651,  1497,   351,
           326,   553,   339,  1139,    13,   366,    40,   836,   470,   765,
           284,  5938,   345,    13,   314,   765,   345,   284,   760,   326,
           314,  1101,   994,   284,  1805,   345,    11,   290,   326,   345]])


# GPT Chatbot

In [9]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
def generate_response(prompt, model, tokenizer, max_length=50, temperature=0.7, top_k=50, top_p=0.95):
    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    output = model.generate(
        input_ids,
        max_length=max_length,
        temperature=temperature,
        top_k=top_k,
        top_p=top_p,
        pad_token_id=tokenizer.eos_token_id
    )
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    return response
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)
print("Chatbot: Hello! Type 'exit' to end the conversation.")
while True:
    user_input = input("You: ")
    if user_input.lower() == 'exit':
        print("Chatbot: Goodbye!")
        break
    chatbot_response = generate_response(user_input, model, tokenizer)
    print("Chatbot:", chatbot_response)


Chatbot: Hello! Type 'exit' to end the conversation.
You: exit
Chatbot: Goodbye!


# roBERTa
RoBERTa is a transformer-based language model introduced by Facebook AI Research (FAIR). It is an extension and optimization of BERT (Bidirectional Encoder Representations from Transformers). RoBERTa addresses some limitations of BERT and incorporates various modifications and improvements, resulting in a robust and high-performance model for natural language understanding tasks

In [10]:
from transformers import RobertaTokenizer, RobertaModel
import torch
tokenizer=RobertaTokenizer.from_pretrained("roberta-base")
model=RobertaModel.from_pretrained("roberta-base")
text="Natural language processing helps to make machines to understand human language"
input_ids=tokenizer.encode(text,return_tensors="pt")
output=model(input_ids)
print(output.last_hidden_state)

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tensor([[[-0.0335,  0.0839, -0.0069,  ..., -0.0631, -0.0782,  0.0077],
         [ 0.0310,  0.2126,  0.1036,  ..., -0.1349, -0.0775,  0.2646],
         [-0.0922, -0.0191,  0.1473,  ...,  0.2454, -0.1423,  0.0543],
         ...,
         [ 0.0792, -0.0156,  0.4738,  ...,  0.1066, -0.4978,  0.3638],
         [-0.1973,  0.0195,  0.2303,  ...,  0.1483, -0.2062,  0.1375],
         [-0.0309,  0.0876, -0.0225,  ..., -0.1057, -0.0967, -0.0178]]],
       grad_fn=<NativeLayerNormBackward0>)


# Text Classification

In [13]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from torch import nn
import torch

# Load pre-trained RoBERTa tokenizer and model for sequence classification
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForSequenceClassification.from_pretrained('roberta-base')

# Example input text for classification
text = "Hugging Face Transformers is great for text classification!"

# Tokenize input text
input_ids = tokenizer.encode(text, return_tensors='pt')

# Forward pass through the model
outputs = model(input_ids)

# Extract the logits (raw scores) from the output
logits = outputs.logits

# Apply softmax to get probabilities
probabilities = nn.functional.softmax(logits, dim=1)

# Get the predicted class index
predicted_class = torch.argmax(probabilities, dim=1).item()

# Print the predicted class label and associated probability
labels = model.config.id2label
predicted_label = labels[predicted_class]
predicted_probability = probabilities[0, predicted_class].item()

print(f"Predicted Class: {predicted_label}")
print(f"Predicted Probability: {predicted_probability:.4f}")


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Predicted Class: LABEL_1
Predicted Probability: 0.5352


# Fine tuning

In [14]:
import torch
from torch.utils.data import DataLoader, TensorDataset
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)
texts = ["I love Hugging Face Transformers!", "I dislike fine-tuning."]
labels = [1, 0]
tokenized_texts = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
dataset = TensorDataset(tokenized_texts["input_ids"], tokenized_texts["attention_mask"], torch.tensor(labels))
loader = DataLoader(dataset, batch_size=2, shuffle=True)
optimizer = AdamW(model.parameters(), lr=1e-5)
num_epochs = 3

# Fine-tuning loop
for epoch in range(num_epochs):
    model.train()
    for batch in loader:
        optimizer.zero_grad()
        inputs, attention_mask, labels = batch
        outputs = model(inputs, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
model.save_pretrained("fine_tuned_model")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


# Inference